# AWS Organizations Examples

In [2]:
# First import the library
import aws_organizations as aws_orgs
# We need boto_session_manager to set the credential
from boto_session_manager import BotoSesManager
# rich is just for pretty printing
from rich import print as rprint

The ``aws_orgs.OrgStructure.get_org_structure`` method will call AWS API to do the following:

- get the current organization and it's root account and root id
- use root id to get all the accounts and organization units tree structure recursively
- create a in-memory ``TREE`` data structure for better user experience

In [3]:
bsm = BotoSesManager(profile_name="my-org_infra_us_east_1")
org_struct = aws_orgs.OrgStructure.get_org_structure(bsm)

In [4]:
# You can visualize the tree structure
print(org_struct.visualize())

root (Root 'o-igflpr8b78')
├── app (OrgUnit 'ou-rkp6-vq6m3h5y')
│   └── my-org-app-dev (Account '222222222222')
├── infra (OrgUnit 'ou-rkp6-cxgi4leg')
│   └── my-org-infra (Account '333333333333')
├── sandbox (OrgUnit 'ou-rkp6-r7cuoq1v')
├── ml (OrgUnit 'ou-rkp6-s4uirmja')
│   ├── my-org-ml-dev (Account '444444444444')
│   ├── my-org-ml-staging (Account '555555555555')
│   └── my-org-ml-prod (Account '666666666666')
└── my-org-root (Account '111111111111')


In [18]:
# You can also dump the organization structure data to CSV
csv = org_struct.to_csv()

import io
import polars as pl

pl.Config.set_tbl_rows(100)
df = pl.read_csv(io.StringIO(csv), sep="\t")
print(df)

shape: (11, 5)
┌─────────┬───────────────────────────────┬──────────────────┬──────────────────┬────────┐
│ Type    ┆ Path                          ┆ Id               ┆ ParentId         ┆ RootId │
│ ---     ┆ ---                           ┆ ---              ┆ ---              ┆ ---    │
│ str     ┆ str                           ┆ str              ┆ str              ┆ str    │
╞═════════╪═══════════════════════════════╪══════════════════╪══════════════════╪════════╡
│ Root    ┆ root                          ┆ o-igflpr8b78     ┆ None             ┆ r-rkp6 │
│ OrgUnit ┆ root | app                    ┆ ou-rkp6-vq6m3h5y ┆ o-igflpr8b78     ┆ r-rkp6 │
│ Account ┆ root | app | my-org-app-dev   ┆ 222222222222     ┆ ou-rkp6-vq6m3h5y ┆ r-rkp6 │
│ OrgUnit ┆ root | infra                  ┆ ou-rkp6-cxgi4leg ┆ o-igflpr8b78     ┆ r-rkp6 │
│ Account ┆ root | infra | my-org-infra   ┆ 333333333333     ┆ ou-rkp6-cxgi4leg ┆ r-rkp6 │
│ OrgUnit ┆ root | sandbox                ┆ ou-rkp6-r7cuoq1v ┆ o-igflpr8b78

In [5]:
# You can access the top level accounts
rprint(org_struct.root.accounts)

[
    Account(
        id='111111111111',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/111111111111',
        name='my-org-root',
        email='my.name+my-org-root@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 5, 15, 52, 58, 47000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    )
]

In [6]:
# You can access the top level organizational units
rprint(org_struct.root.org_units)

[
    OrganizationalUnit(
        id='ou-rkp6-vq6m3h5y',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-vq6m3h5y',
        name='app',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-cxgi4leg',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-cxgi4leg',
        name='infra',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-r7cuoq1v',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-r7cuoq1v',
        name='sandbox',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-s4uirmja',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-s4uirmja',
        name='ml',
        root_id='r-rkp6'
    )
]

In [7]:
# You can access all accounts recursively
rprint(org_struct.root.all_accounts)

[
    Account(
        id='222222222222',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/222222222222',
        name='my-org-app-dev',
        email='my.name+my-org-app-dev@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 14, 10, 55, 461000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='333333333333',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/333333333333',
        name='my-org-infra',
        email='my.name+my-org-infra@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 7, 23, 26, 48, 257000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='444444444444',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/444444444444',
        name='my-org-ml-dev',
        email='my.name+my-org-ml-dev@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 5, 23, 10, 17, 816000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='555555555555',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/555555555555',
        name='my-org-ml-staging',
        email='my.name+my-org-ml-staging@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 34, 947000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='666666666666',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/666666666666',
        name='my-org-ml-prod',
        email='my.name+my-org-ml-prod@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 54, 519000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='111111111111',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/111111111111',
        name='my-org-root',
        email='my.name+my-org-root@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 5, 15, 52, 58, 47000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    )
]

In [8]:
# You can access all organizational units recursively
rprint(org_struct.root.all_org_units)

[
    OrganizationalUnit(
        id='ou-rkp6-vq6m3h5y',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-vq6m3h5y',
        name='app',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-cxgi4leg',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-cxgi4leg',
        name='infra',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-r7cuoq1v',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-r7cuoq1v',
        name='sandbox',
        root_id='r-rkp6'
    ),
    OrganizationalUnit(
        id='ou-rkp6-s4uirmja',
        arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-s4uirmja',
        name='ml',
        root_id='r-rkp6'
    )
]

In [9]:
# You can get a node (account or org unit) by id
# this is an org unit
ou_ml = org_struct.get_node_by_id("ou-rkp6-s4uirmja")
rprint(ou_ml)

ml (OrgUnit 'ou-rkp6-s4uirmja')

In [10]:
# You can also get a node (account or org unit) by name
ou_ml = org_struct.get_node_by_name("ml")
rprint(ou_ml)

ml (OrgUnit 'ou-rkp6-s4uirmja')

In [11]:
rprint(ou_ml.obj)

OrganizationalUnit(
    id='ou-rkp6-s4uirmja',
    arn='arn:aws:organizations::111111111111:ou/o-igflpr8b78/ou-rkp6-s4uirmja',
    name='ml',
    root_id='r-rkp6'
)

In [16]:
# you can access top level accounts in an OU
rprint(ou_ml.accounts)

[
    Account(
        id='444444444444',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/444444444444',
        name='my-org-ml-dev',
        email='my.name+my-org-ml-dev@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 5, 23, 10, 17, 816000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='555555555555',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/555555555555',
        name='my-org-ml-staging',
        email='my.name+my-org-ml-staging@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 34, 947000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='666666666666',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/666666666666',
        name='my-org-ml-prod',
        email='my.name+my-org-ml-prod@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 54, 519000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    )
]

In [17]:
# you can also do it recursively
rprint(ou_ml.all_accounts)

[
    Account(
        id='444444444444',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/444444444444',
        name='my-org-ml-dev',
        email='my.name+my-org-ml-dev@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 5, 23, 10, 17, 816000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='555555555555',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/555555555555',
        name='my-org-ml-staging',
        email='my.name+my-org-ml-staging@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 34, 947000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    ),
    Account(
        id='666666666666',
        arn='arn:aws:organizations::111111111111:account/o-igflpr8b78/666666666666',
        name='my-org-ml-prod',
        email='my.name+my-org-ml-prod@mymail.com',
        status='ACTIVE',
        joined_method='INVITED',
        joined_timestamp=datetime.datetime(2023, 3, 6, 10, 46, 54, 519000, tzinfo=tzlocal()),
        root_id='r-rkp6'
    )
]

In [15]:
# you can access top level organizational units in an OU
rprint(ou_ml.org_units)

[]

In [14]:
# you can also do it recursively
rprint(ou_ml.all_org_units)

[]

It might be time consuming (5-10 seconds) to recursively get all information via AWS API. You can serialize the results to JSON, cache it and read it back (0.01 seconds)

In [20]:
# serialize
data = org_struct.serialize() # put this data in JSON, you can use ``jsonpickle.dumps``

# deserialize
org_struct = aws_orgs.OrgStructure.deserialize(data)